<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_11_24_Multi_turn_Analysis_of_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to use Graphlit to perform multi-step analysis of provided image.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [18]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [19]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [20]:
from typing import List, Optional

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def revise_image(prompt: str, uri: str, specification_id: str, conversation_id: Optional[str] = None):
    if graphlit.client is None:
        return None, None

    try:
        response = await graphlit.client.revise_image(prompt=prompt, uri=uri, id=conversation_id, specification=input_types.EntityReferenceInput(id=specification_id))

        message = response.revise_image.message.message if response.revise_image is not None and response.revise_image.message is not None else None
        conversation_id = response.revise_image.conversation.id if response.revise_image is not None and response.revise_image.conversation is not None else None

        return message, conversation_id
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None, None

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)


Execute Graphlit example

In [21]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()

print('Deleted all conversations and specifications.')

uri = "https://graphlitplatform.blob.core.windows.net/test/images/2657-Emerging-LLM-App-Stack-R2-1-of-4-2.png"

Deleted all conversations and specifications.


Create specification, and start multi-turn summarization conversation.

In [22]:
conversation_id = None

specification_id = await create_openai_specification(enums.OpenAIModels.GPT4O_128K)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    prompt = "Describe this for me."

    message, conversation_id = await revise_image(prompt, uri, specification_id)

    if message is not None:
        display(Markdown(f'**Revision:**\n{message}'))
        print()


Created specification [235343ed-46a8-46a6-8c91-03c7a66da1f2].


**Revision:**
The Emerging LLM App Stack diagram illustrates the components and data flow involved in building applications with large language models (LLMs).

Key components include data pipelines, embedding models, vector databases, orchestration, APIs/plugins, LLM cache, logging/LLMops, validation, and app hosting.

Data pipelines process contextual data, which is then used by embedding models to create vector representations stored in vector databases.

Orchestration tools manage the flow of data and interactions between components, integrating with APIs and plugins to enhance functionality.

LLM APIs and hosting options include proprietary APIs, open APIs, cloud providers, and opinionated cloud services, offering flexibility in deployment.

The diagram uses arrows to indicate the flow of data: contextual data, prompts, user queries, and outputs, highlighting the interaction between components.

In [23]:
    prompt = "OK, not bad, but make it more detailed. Infer anything you can from the intent of the image."

    message, conversation_id = await revise_image(prompt, uri, specification_id, conversation_id)

    if message is not None:
        display(Markdown(f'**Revision:**\n{message}'))
        print()


**Revision:**
The Emerging LLM App Stack diagram provides a comprehensive overview of the architecture and components involved in developing applications using large language models. It highlights the flow of data from contextual sources through various processing stages to deliver outputs to users. The stack is designed to integrate multiple tools and systems, ensuring flexibility and scalability in application development.

Key components include data pipelines, which handle the ingestion and processing of contextual data, and embedding models that transform this data into vector representations. These vectors are stored in vector databases, facilitating efficient retrieval and manipulation. Orchestration tools manage the interactions between components, ensuring seamless data flow and integration with external APIs and plugins.

The diagram also outlines the role of LLM APIs and hosting options, which provide the necessary infrastructure for deploying and scaling applications. These options include proprietary APIs, open APIs, and various cloud services, offering developers a range of choices based on their specific needs and preferences. The stack is designed to support robust application development, leveraging the capabilities of large language models.

In [24]:
    prompt = "Also extract all visible text and format as Markdown, in addition to a Markdown summary of the image."

    message, conversation_id = await revise_image(prompt, uri, specification_id, conversation_id)

    if message is not None:
        display(Markdown(f'**Revision:**\n{message}'))
        print()


**Revision:**
### Emerging LLM App Stack

- **Contextual data**: Data Pipelines (Databricks, Airflow, Unstructured, ...)
- **Embedding Model**: (OpenAI, Cohere, Hugging Face)
- **Vector Database**: (Pinecone, Weaviate, Chroma, pgvector)
- **Playground**: (OpenAI, nat.dev, Humanloop)
- **Orchestration**: (Python/DIY, LangChain, LlamaIndex, ChatGPT)
- **APIs/Plugins**: (Serp, Wolfram, Zapier, ...)
- **LLM Cache**: (Redis, SQLite, GPTCache)
- **Logging/LLMops**: (Weights & Biases, MLflow, PromptLayer, Helicone)
- **Validation**: (Guardrails, Rebuff, Guidance, LMQL)
- **App Hosting**: (Vercel, Steamship, Streamlit, Modal)

### LLM APIs and Hosting

- **Proprietary API**: (OpenAI, Anthropic)
- **Open API**: (Hugging Face, Replicate)
- **Cloud Provider**: (AWS, GCP, Azure, Coreweave)
- **Opinionated Cloud**: (Databricks, Anyscale, Mosaic, Modal, Runpod, ...)

### Legend

- **Gray boxes**: Key components of the stack, with leading tools/systems listed
- **Arrows**: Show the flow of data through the stack
  - **Dashed arrows**: Contextual data provided by app developers to condition LLM outputs
  - **Red arrows**: Prompts and few-shot examples that are sent to the LLM
  - **Blue arrows**: Queries submitted by users
  - **Red arrows**: Output returned to users

The diagram illustrates the architecture of an emerging LLM app stack, detailing the flow of data from contextual sources through various processing stages to deliver outputs. It highlights key components such as data pipelines, embedding models, and vector databases, which work together to process and store data efficiently. Orchestration tools manage these interactions, integrating with APIs and plugins to enhance functionality and scalability.